In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append(sys.path[0].replace('/Development Notebooks',''))
from subproptools import qtaimExtract as qt
from subproptools import subreor as sr

#the previous code used a distance check, assuming that the two closest nuclei were the ones the CC
#was bonded too
#the problem - with the longer As-C bond, and the CC being so close to As, the CC found that an H,
#not the C was closer to the CC.
#Fixed by instead looping over all atoms to find if it is on the line connecting any atoms

In [2]:
negXAtom = 2
originAtom = 1
sumFileNoExt = 'SubH_AsO-C6H5-2_wb97xd_aug-cc-pvtz'
sumFile = open('SubH_AsO-C6H5-2_wb97xd_aug-cc-pvtz'+".sum","r")
data = sumFile.readlines()
sumFile.close()

In [3]:
atomDict = qt.get_atomic_props(data)

In [4]:
molecule_xyz = qt.get_xyz(sumFileNoExt)

In [5]:
negXAtomLabel = molecule_xyz['Atoms'][negXAtom-1]
attachedAtom = molecule_xyz['Atoms'][originAtom-1]

In [6]:
molecule_orig = sr._set_origin(molecule_xyz['xyz'],originAtom)
molecule_xaxis = sr._set_xaxis(molecule_orig,negXAtom)

In [9]:
#finding posypoint - from _get_posy_point
ccProps = qt.get_cc_props(sumFileNoExt,attachedAtom)
len(ccProps)
vscc = qt.identify_vscc(ccProps,atomDict,attachedAtom)

In [10]:
vscc # here is the problem, CC 4 and 5 should not be found as VSCC
#fixed

{}

In [ ]:
# in identify_vscc
nucDistList = []
potentialccList = []
# we'll look specifically at 4 and 5
cc = 4

In [ ]:
ccProps[cc]['distFromNuc'] > 0.1
multiccDict = ccProps
ccXYZ = ccProps[cc]['xyz']
nucPair = qt._find_closest_nuclei(ccXYZ,atomDict)

In [ ]:
qt._is_on_line(atomDict[nucPair[0]]['xyz'],atomDict[nucPair[1]]['xyz'],ccXYZ)
#should return true, see false

False

In [ ]:
lineStartList = atomDict[nucPair[0]]['xyz']
lineEndList = atomDict[nucPair[1]]['xyz']
pointToCheckList = ccXYZ

In [ ]:
epsilon=0.1

In [ ]:
t = ((pointToCheckList[0]-lineStartList[0])*(lineEndList[0]-lineStartList[0]) + 
        (pointToCheckList[1]-lineStartList[1])*(lineEndList[1]-lineStartList[1]) + 
        (pointToCheckList[2]-lineStartList[2])*(lineEndList[2]-lineStartList[2]))/(((lineEndList[2]-lineStartList[2])**2) + ((lineEndList[1]-lineStartList[1])**2)+((lineEndList[0]-lineStartList[0])**2))
perpPoint = np.array([lineStartList[0]+t*(lineEndList[0]-lineStartList[0]),
                          lineStartList[1]+t*(lineEndList[1]-lineStartList[1]),
                          lineStartList[2]+t*(lineEndList[2]-lineStartList[2])])

In [ ]:
qt._get_dist(pointToCheckList,perpPoint) 

0.43555209623870106

In [ ]:
lineStartList

array([-0.05320803, -2.14407859, -1.00411451])

In [ ]:
lineEndList

array([ 0.02535549, -2.26379136, -3.86799038])

In [ ]:
pointToCheckList

array([ 0.30557456, -1.8970071 , -0.92076369])

In [ ]:
((pointToCheckList[0]-lineStartList[0])*(lineEndList[0]-lineStartList[0]) + 
        (pointToCheckList[1]-lineStartList[1])*(lineEndList[1]-lineStartList[1]) + 
        (pointToCheckList[2]-lineStartList[2])*(lineEndList[2]-lineStartList[2]))/(((lineEndList[2]-lineStartList[2])**2) + ((lineEndList[1]-lineStartList[1])**2)+((lineEndList[0]-lineStartList[0])**2))
atomDict.keys()

dict_keys(['As1', 'H2', 'C3', 'C4', 'C5', 'C6', 'H7', 'C8', 'H9', 'C10', 'H11', 'H12', 'H13', 'C14', 'C15', 'C16', 'C17', 'H18', 'C19', 'H20', 'C21', 'H22', 'H23', 'H24', 'O25'])

# the issue is in finding the connected atoms - those CCs should probably be for the carbons, but the hydrogen is actually closer. So it finds the perpendicular point between the As and the H instead, and then checks distance from that to the As-C CC, which obviously is non-zero